# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4879, uptime 0:00:22
mini_beamline                    RUNNING   pid 4880, uptime 0:00:22
random_walk                      RUNNING   pid 4881, uptime 0:00:22
random_walk_horiz                RUNNING   pid 4882, uptime 0:00:22
random_walk_vert                 RUNNING   pid 4883, uptime 0:00:22
simple                           RUNNING   pid 4884, uptime 0:00:22
thermo_sim                       RUNNING   pid 4885, uptime 0:00:22
trigger_with_pc                  RUNNING   pid 4886, uptime 0:00:22


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-18 14:39:32
Persistent Unique Scan ID: 'ed56ae40-f5e5-491c-b983-1f4dd1caeb39'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 14:39:32.8 |     -1.000 |     102808 |
|         2 | 14:39:32.8 |     -0.500 |     103384 |
|         3 | 14:39:32.8 |      0.000 |     104818 |
|         4 | 14:39:32.8 |      0.500 |     103802 |
|         5 | 14:39:32.8 |      1.000 |     102534 |


+-----------+------------+------------+------------+
generator scan ['ed56ae40'] (scan num: 1)





('ed56ae40-f5e5-491c-b983-1f4dd1caeb39',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/ed56ae40-f5e5-491c-b983-1f4dd1caeb39-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-18 14:39:33
Persistent Unique Scan ID: 'c8834de5-355b-4472-a675-b6618d412706'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 14:39:33.3 |          -1 | 1111600724 |


|         2 | 14:39:33.5 |          -0 | 1105978357 |


|         3 | 14:39:33.6 |           0 | 1093442834 |


|         4 | 14:39:33.7 |           0 | 1087096851 |


|         5 | 14:39:33.8 |           1 | 1079349184 |


+-----------+------------+-------------+------------+
generator scan ['c8834de5'] (scan num: 2)





('c8834de5-355b-4472-a675-b6618d412706',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 14:39:34
Persistent Unique Scan ID: '0380aa5f-a8a1-4419-802d-b65a02b470ca'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 14:39:34.4 | 1056886313 |


|         2 | 14:39:35.3 | 1076318280 |


|         3 | 14:39:36.3 | 1128310460 |


...

|        28 | 14:40:01.3 | 1105776643 |


|        29 | 14:40:02.3 | 1055439699 |


|        30 | 14:40:03.4 | 1079784202 |


+-----------+------------+------------+
generator count ['0380aa5f'] (scan num: 3)





('0380aa5f-a8a1-4419-802d-b65a02b470ca',)